<a href="https://colab.research.google.com/github/srujith7/AD-075/blob/main/textclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
dataset=pd.read_csv("/content/hate_speech.csv")
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
dataset.shape

(5242, 3)

In [3]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [4]:
for index,tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [9]:
import re
def clean_text(text):
  text = re.sub(r'[^a-zA-Z\']', ' ', text)
  text = re.sub(r'[^\x00-\x7F]+', '', text)
  text =  text.lower()
  return text

In [8]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [10]:
dataset.head(3)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty


In [11]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words=stopwords.words('english')
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [14]:
def gen_freq(text):
  word_list=[]
  for word in text.split():
    word_list.extend(word)
  word_freq=pd.Series(word_list).value_counts()
  word_freq=word_freq.drop(stop_words,errors='ignore')
  return word_freq

In [16]:
def any_neg(words):
  for word in words:
    if word in ['no','nor','not'] or re.search:
      return 1
  else:
    return 0